In [ ]:
import tkinter as tk
from tkinter import messagebox
import numpy as np
import joblib
import os
from catboost import CatBoostRegressor

# Load the catb1 model
model_path_catb1 = "C:/Users/asus1/desktop/catb1.joblib"
if os.path.exists(model_path_catb1):
    model_catb1 = joblib.load(model_path_catb1)
else:
    messagebox.showerror("Error", "catb1 model file not found.")
    exit()

# Load the catb2 model
model_path_catb2 = "C:/Users/asus1/desktop/catb2.joblib"
if os.path.exists(model_path_catb2):
    model_catb2 = joblib.load(model_path_catb2)
else:
    messagebox.showerror("Error", "catb2 model file not found.")
    exit()

# Create the Tkinter window
root = tk.Tk()
root.title("Prediction of first peak load and failure load")
root.configure(bg='white')  # Set the background color of the window to white

# Create a canvas widget
canvas = tk.Canvas(root, width=1000, height=450, bg='white')  # Set the background color of the canvas to white
canvas.pack()

# Function to create labels on the canvas
def create_label(text, font, fg, bg, x, y):
    label = tk.Label(root, text=text, font=font, fg=fg, bg=bg)
    canvas.create_window(x, y, anchor="w", window=label)
    return label

# Define the GUI elements
label_inputdefinetitle2 = create_label('Prediction of first peak and failure loads', ('Palatino Linotype', 18, 'bold', 'underline'), '#0000FF', 'white', 35,10)
label_inputdefinetitle = create_label('Definition of Parameters', ('Palatino Linotype', 16, 'bold', 'italic'), '#000000', 'white', 10,45)
param_labels = [
    'X1: Compressive strength of the standard cylinder (fo|MPa)',
    'X2: Diameter of the internal hole (Do|mm)',
    'X3: Outer diameter of hollow concrete column (D|mm)',
    'X4: Column’s height (H|mm)',
    'X5: Centre to centre spacing of hoop reinforcing bars (S|mm)',
    'X6: Area of FRP reinforcing hoop bar (Ab|mm^2)',
    'X7: Tensile strength of FRP hoop bar (fh|MPa)',
    'X8: Ratio of longitudinal FRP reinforcing bars multiplied by its tensile strength (ρh*fv|MPa)',
   ]
for i, text in enumerate(param_labels, start=1):
    create_label(text, ('Georgia', 13), '#006600', 'white', 10, 50 + i*30)

label_inputs = create_label('Inputs', ('Palatino Linotype', 16, 'bold', 'italic'), '#000000', 'white', 800, 45)

entry_fields = ['X1', 'X2', 'X3', 'X4','X5', 'X6', 'X7', 'X8']
entries = {}
for i, field in enumerate(entry_fields, start=1):
    create_label(f'{field}: ', ('Georgia', 14), '#FF0000', 'white', 780, 45+ i*30)
    entry = tk.Entry(root)
    canvas.create_window(900, 45 + i*30, window=entry)
    entries[field] = entry

# Output Section
label_output = create_label('Outputs', ('Palatino Linotype', 16, 'bold'), '#000000', 'white', 20, 340)

label_result_catb1 = tk.Label(root, text='First peak load (Pn,1, kN)', font=('Georgia', 14, 'bold'), fg='#8B0000', bg='white')
canvas.create_window(100, 370, anchor="w", window=label_result_catb1)

label_result_catb2 = tk.Label(root, text='Failure load (Pn,2, kN)', font=('Georgia', 14, 'bold'), fg='#8B0000', bg='white')
canvas.create_window(100, 420, anchor="w", window=label_result_catb2)

# Predict button command function
def predict():
    # Collect input data
    input_values = []
    for field in entry_fields:
        try:
            value = float(entries[field].get())
            input_values.append(value)
        except ValueError:
            messagebox.showerror("Error", f"Invalid input for {field}. Please enter a numeric value.")
            return

    # Predict using the loaded models
    input_data = np.array([input_values])
    try:
        prediction_catb1 = model_catb1.predict(input_data)
        prediction_catb2 = model_catb2.predict(input_data)
        
        # Display the predictions
        label_result_catb1.config(text=f'Pn,1 = {prediction_catb1[0]:.4f} kN')
        label_result_catb2.config(text=f'Pn,2 = {prediction_catb2[0]:.4f} kN')
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred during prediction: {e}")

# Predict button
predict_button = tk.Button(root, text='Calculate', font=('Arial Black', 16, 'bold'), command=predict, fg='#FFFFFF', bg='green')
canvas.create_window(820, 400, window=predict_button)

# Run the Tkinter event loop
root.mainloop()
